In [4]:
import cv2
from ultralytics import YOLO
def detectar_camaras():
    index = 0
    arr = []
    while True:
        cap = cv2.VideoCapture(index)
        if not cap.read()[0]:
            break
        else:
            arr.append(index)
        cap.release()
        index += 1
    return arr

# Uso de la función
camaras_disponibles = detectar_camaras()
print("Cámaras disponibles:", camaras_disponibles)

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\Usuario\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Cámaras disponibles: [0]


In [ ]:

# Instancia del modelo YOLO con los pesos especificados 
pesos_yolo = "yolov8n.pt" # Usamos la ruta relativa
model = YOLO(pesos_yolo)

# Verificar si la cámara está disponible
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("No se pudo acceder a la cámara.")
else:
    try:
        # Realiza la detección de objetos en el flujo de video de la cámara del computador 
        results = model(source=0, show=True, conf=0.3, save=True)
    except Exception as e:
        print("Ocurrió un error al realizar la detección de objetos:", e)
    finally:
        cap.release()


1/1: 0... Success  (inf frames of shape 640x480 at 30.00 FPS)


WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

0: 480x640 1 person, 139.1ms
0: 480x640 1 person, 67.2ms
0: 480x640 1 person, 63.8ms
0: 480x640 1 person, 66.9ms
0: 480x640 1 person, 64.9ms
0: 480x640 1 person, 64.3ms
0: 480x640 1 person, 61.5ms
0: 480x640 1 person, 65.5ms
0: 480x640 1 person, 72.3ms
0: 480x640 1 person, 67.4ms
0: 480x640 1 person, 65.3ms
0: 480x640 1 person, 69.7ms
0: 480x640 1 person, 64.5ms
0: 480x640 1 person, 64.4ms
0:

In [ ]:
# Tracking
from ultralytics import YOLO 
# Configure the tracking parameters and run the tracker 
model = YOLO('yolov8n.pt') 
# Seleccionamos la clase 66 (keyboard)
results = model.track(source="0", conf=0.3, iou=0.5, show=True, classes=[66])  


1/1: 0... Success  (inf frames of shape 640x480 at 30.00 FPS)


WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

0: 480x640 (no detections), 62.3ms
0: 480x640 (no detections), 43.4ms
0: 480x640 (no detections), 39.3ms
0: 480x640 (no detections), 38.9ms
0: 480x640 (no detections), 40.4ms
0: 480x640 (no detections), 43.3ms
0: 480x640 (no detections), 44.1ms
0: 480x640 (no detections), 39.5ms
0: 480x640 (no detections), 38.2ms
0: 480x640 (no detections), 41.3ms
0: 480x640 (no detections), 40.2ms
0: 480x640

KeyboardInterrupt: 

WARNING  Video stream unresponsive, please check your IP camera connection.
WARNING  Video stream unresponsive, please check your IP camera connection.


In [ ]:
from sklearn.model_selection import train_test_split
import shutil
import os

dataset_ZIP_path = "Dice.v2-medium-color.yolov8.zip"

dataset_path = "datasets\\"+dataset_ZIP_path[:-4]
if not os.path.exists(dataset_path):
    # Create a directory to store the dataset
    os.makedirs(dataset_path)
    # Unzip the dataset in the created directory
    
    with zipfile.ZipFile(dataset_ZIP_path, "r") as zip_ref:
        zip_ref.extractall(dataset_path)
        

images_path = os.path.join(dataset_path+"\\export", "images")

# Get the list of all image files
image_files = [os.path.join(images_path, f) for f in os.listdir(images_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Split the dataset into train, val, and test sets
train_files, test_files = train_test_split(image_files, test_size=0.2, random_state=42)
train_files, val_files = train_test_split(train_files, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# Function to copy files to the respective directories
def copy_files(file_list, destination_folder):
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
    for file in file_list:
        shutil.copy(file, destination_folder)

# Copy the files to the respective directories
copy_files(train_files, os.path.join(dataset_path, "train/images"))
copy_files(val_files, os.path.join(dataset_path, "valid/images"))
copy_files(test_files, os.path.join(dataset_path, "test/images"))

print("Dataset split into train, val, and test sets.")

Dataset split into train, val, and test sets.


In [9]:
import zipfile
import os
from ultralytics import YOLO
import torch
# Load a model
model = YOLO("yolov8n.pt")






# Train the model
train_results = model.train(
    data=dataset_ZIP_path[:-4] + "\\data.yaml",  # path to dataset YAML
    epochs=100,  # number of training epochs
    imgsz=640,  # training image size
    device= "cuda" if torch.cuda.is_available() else "cpu",  # CPU or GPU
)

# Evaluate model performance on the validation set
metrics = model.val()

def detectObject(path_to_image: str):
    # Perform object detection on an image
    if not os.path.exists(path_to_image):
        print("La imagen no existe.")
        return
    results = model(path_to_image)
    results[0].show()
    return results

for img in os.listdir("prueba/"):
    detectObject("prueba/" + img)


# Export the model to ONNX format
path = model.export(format="onnx")  # return path to exported model

Ultralytics 8.3.86  Python-3.13.2 torch-2.6.0+cpu CPU (11th Gen Intel Core(TM) i5-11400F 2.60GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=Dice.v2-medium-color.yolov8\data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True

100%|██████████| 755k/755k [00:00<00:00, 4.98MB/s]

Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     


  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  8                  -1  1    460288  ultralytics.nn.modules.block.C2f             [256,

train: Scanning E:\github repositories\bigData\1CEIABDTA - M2D\Modelos de Inteligencia artificial\TAREA 6 (24-25) DETECCIÓN DE OBJETOS CON YOLO\datasets\Dice.v2-medium-color.yolov8\train\labels... 0 images, 430 backgrounds, 0 corrupt: 100%|██████████| 430/430 [00:07<00:00, 59.63it/s]

...

                   all        144          0          0          0          0          0
WARNING  no labels found in detect set, can not compute metrics without labels



C:\Users\Usuario\AppData\Roaming\Python\Python313\site-packages\ultralytics\utils\metrics.py:619: RuntimeWarning: Mean of empty slice.
  i = smooth(f1_curve.mean(0), 0.1).argmax()  # max F1 index
C:\Users\Usuario\AppData\Roaming\Python\Python313\site-packages\numpy\_core\_methods.py:139: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      2/100         0G          0      105.4          0          0        640:  78%|███████▊  | 21/27 [02:21<00:40,  6.73s/it]


KeyboardInterrupt: 